# Cue Reward Association 

In this simple meta-learning task, one of four input cues is arbitrarily chosen as a *target cue*. The agent is repeatedly shown two random cues in succession, and then a *response cue* during which the agent must respond with a 1 if the target was part of the pair, or 0 otherwise. 


**Google Drive Set-up:**


In [1]:
# this mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# enter the foldername in your Drive where you have saved the unzipped
# project folder, e.g. 'project submissions/A09_A10_A54'
# $$ for us - 'DSc Term Project/'
FOLDERNAME = 'DSc Term Project/Task 1'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

# test - this notebooks name should show up:
# is oserror - restart runtime
%cd /content/drive/My\ Drive/$FOLDERNAME
%ls 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1khPuOHY9jnXOARox2UYYUJ3TRirri2cS/DSc Term Project/Task 1
'Cue Reward Association.ipynb'   RM.ipynb          SM_rudraksh.ipynb
 NM.ipynb                        RM_theirs.ipynb   SM_theirs.ipynb
 NP.ipynb                        saved/            utils/
'Plot Test.ipynb'                SM.ipynb


**Import Statements:**


In [0]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch import optim
from torch.optim import lr_scheduler

import numpy as np
import time
import pickle
import random
from tqdm.autonotebook import tqdm
#from graphics import *

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (20.0, 16.0) # set default size of plots



np.set_printoptions(precision=4)

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

**GPU Set-up:**

You have an option to use GPU by setting the flag to `True` below.

The global variables `dtype` and `device` will control the data types throughout this notebook. 

You need to manually switch to a GPU device. You can do this by clicking `Runtime -> Change runtime type` and selecting `GPU` under `Hardware Accelerator`. Note that you have to rerun the cells from the top since the kernel gets restarted upon switching runtimes.

In [3]:
# flag
USE_GPU = True

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
    dtype = torch.float32
else:
    device = torch.device('cpu')
    dtype = torch.float32

print('Using Device:', device)

Using Device: cuda


In [0]:
import pdb
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import torch.nn.functional as F




##ttype = torch.FloatTensor;
#ttype = torch.cuda.FloatTensor;

#ttype = torch.FloatTensor;
#ttype = torch.cuda.FloatTensor;



class NonPlasticRNN(nn.Module):
    def __init__(self, params):
        super(NonPlasticRNN, self).__init__()
        # NOTE: 'outputsize' excludes the value and neuromodulator outputs!
        for paramname in ['outputsize', 'inputsize', 'hs', 'bs', 'fm']:
            if paramname not in params.keys():
                raise KeyError("Must provide missing key in argument 'params': "+paramname)
        NBDA = 1 # For now we limit the number of neuromodulatory-output neurons to 1
        # Doesn't work with our version of PyTorch:
        #self.device = torch.device("cuda:0" if self.params['device'] == 'gpu' else "cpu")
        self.params = params
        self.activ = F.tanh
        self.i2h = torch.nn.Linear(self.params['inputsize'], params['hs'])
        self.w =  torch.nn.Parameter((.01 * torch.t(torch.rand(params['hs'], params['hs']))), requires_grad=True)
        self.h2o = torch.nn.Linear(params['hs'], self.params['outputsize'])
        self.h2v = torch.nn.Linear(params['hs'], 1)


    def forward(self, inputs, hidden): #, hebb):
        BATCHSIZE = self.params['bs']
        HS = self.params['hs']

        # Here, the *rows* of w and hebb are the inputs weights to a single neuron
        # hidden = x, hactiv = y
        hactiv = self.activ(self.i2h(inputs).view(BATCHSIZE, HS, 1) + torch.matmul(self.w,
                        hidden.view(BATCHSIZE, HS, 1))).view(BATCHSIZE, HS)
        #hactiv = self.activ(self.i2h(inputs).view(BATCHSIZE, HS, 1) + torch.matmul((self.w + torch.mul(self.alpha, hebb)),
        #                hidden.view(BATCHSIZE, HS, 1))).view(BATCHSIZE, HS)
        activout = self.h2o(hactiv)  # Pure linear, raw scores - will be softmaxed by the calling program
        valueout = self.h2v(hactiv)

        hidden = hactiv

        return activout, valueout, hidden #, hebb


    def initialZeroState(self):
        BATCHSIZE = self.params['bs']
        return Variable(torch.zeros(BATCHSIZE, self.params['hs']), requires_grad=False )





class PlasticRNN(nn.Module):
    def __init__(self, params):
        super(PlasticRNN, self).__init__()
        # NOTE: 'outputsize' excludes the value and neuromodulator outputs!
        for paramname in ['outputsize', 'inputsize', 'hs', 'bs', 'fm']:
            if paramname not in params.keys():
                raise KeyError("Must provide missing key in argument 'params': "+paramname)
        NBDA = 1 # For now we limit the number of neuromodulatory-output neurons to 1
        # Doesn't work with our version of PyTorch:
        #self.device = torch.device("cuda:0" if self.params['device'] == 'gpu' else "cpu")
        self.params = params
        self.activ = F.tanh
        self.i2h = torch.nn.Linear(self.params['inputsize'], params['hs'])
        self.w =  torch.nn.Parameter((.01 * torch.t(torch.rand(params['hs'], params['hs']))), requires_grad=True)
        self.alpha =  torch.nn.Parameter((.01 * torch.t(torch.rand(params['hs'], params['hs']))), requires_grad=True)
        self.eta = torch.nn.Parameter((.1 * torch.ones(1)), requires_grad=True)  # Everyone has the same eta
        #self.h2DA = torch.nn.Linear(params['hs'], NBDA)
        self.h2o = torch.nn.Linear(params['hs'], self.params['outputsize'])
        self.h2v = torch.nn.Linear(params['hs'], 1)

    def forward(self, inputs, hidden, hebb):
        BATCHSIZE = self.params['bs']
        HS = self.params['hs']

        # Here, the *rows* of w and hebb are the inputs weights to a single neuron
        # hidden = x, hactiv = y
        hactiv = self.activ(self.i2h(inputs).view(BATCHSIZE, HS, 1) + torch.matmul((self.w + torch.mul(self.alpha, hebb)),
                        hidden.view(BATCHSIZE, HS, 1))).view(BATCHSIZE, HS)
        activout = self.h2o(hactiv)  # Pure linear, raw scores - will be softmaxed by the calling program
        valueout = self.h2v(hactiv)

        # Now computing the Hebbian updates...

        # deltahebb has shape BS x HS x HS
        # Each row of hebb contain the input weights to a neuron
        deltahebb =  torch.bmm(hactiv.view(BATCHSIZE, HS, 1), hidden.view(BATCHSIZE, 1, HS)) # batched outer product...should it be other way round?
        hebb = torch.clamp(hebb + self.eta * deltahebb, min=-1.0, max=1.0)

        hidden = hactiv

        return activout, valueout, hidden, hebb

    def initialZeroHebb(self):
        return Variable(torch.zeros(self.params['bs'], self.params['hs'], self.params['hs']) , requires_grad=False)

    def initialZeroState(self):
        BATCHSIZE = self.params['bs']
        return Variable(torch.zeros(BATCHSIZE, self.params['hs']), requires_grad=False )




class SimpleModulRNN(nn.Module):
    def __init__(self, params):
        super(SimpleModulRNN, self).__init__()
        # NOTE: 'outputsize' excludes the value and neuromodulator outputs!
        for paramname in ['outputsize', 'inputsize', 'hs', 'bs', 'fm']:
            if paramname not in params.keys():
                raise KeyError("Must provide missing key in argument 'params': "+paramname)
        NBDA = 1 # For now we limit the number of neuromodulatory-output neurons to 1
        # Doesn't work with our version of PyTorch:
        #self.device = torch.device("cuda:0" if self.params['device'] == 'gpu' else "cpu")
        self.params = params
        self.activ = F.tanh
        self.i2h = torch.nn.Linear(self.params['inputsize'], params['hs'])
        self.w =  torch.nn.Parameter((.01 * torch.t(torch.rand(params['hs'], params['hs']))), requires_grad=True)
        self.alpha =  torch.nn.Parameter((.01 * torch.t(torch.rand(params['hs'], params['hs']))), requires_grad=True)
        self.eta = torch.nn.Parameter((.1 * torch.ones(1)), requires_grad=True)  # Everyone has the same eta (only for the non-modulated part, if any!)
        self.h2DA = torch.nn.Linear(params['hs'], NBDA)
        self.h2o = torch.nn.Linear(params['hs'], self.params['outputsize'])
        self.h2v = torch.nn.Linear(params['hs'], 1)

    def forward_test(self, inputs, hidden, hebb):
        NBDA = 1
        BATCHSIZE = self.params['bs']
        HS = self.params['hs']
        hactiv = self.activ(self.i2h(inputs).view(BATCHSIZE, HS, 1) + torch.matmul(self.w,
                        hidden.view(BATCHSIZE, HS, 1))).view(BATCHSIZE, HS)
        activout = self.h2o(hactiv)  # Pure linear, raw scores - will be softmaxed by the calling program
        valueout = self.h2v(hactiv)
        return activout, valueout, 0, hidden, hebb

    def forward(self, inputs, hidden, hebb):
        NBDA = 1
        BATCHSIZE = self.params['bs']
        HS = self.params['hs']

        # Here, the *rows* of w and hebb are the inputs weights to a single neuron
        # hidden = x, hactiv = y
        hactiv = self.activ(self.i2h(inputs).view(BATCHSIZE, HS, 1) + torch.matmul((self.w + torch.mul(self.alpha, hebb)),
                        hidden.view(BATCHSIZE, HS, 1))).view(BATCHSIZE, HS)
        activout = self.h2o(hactiv)  # Pure linear, raw scores - will be softmaxed by the calling program
        valueout = self.h2v(hactiv)

        # Now computing the Hebbian updates...

        # With batching, DAout is a matrix of size BS x 1 (Really BS x NBDA, but we assume NBDA=1 for now in the deltahebb multiplication below)
        if self.params['da'] == 'tanh':
            DAout = F.tanh(self.h2DA(hactiv))
        elif self.params['da'] == 'sig':
            DAout = F.sigmoid(self.h2DA(hactiv))
        elif self.params['da'] == 'lin':
            DAout =  self.h2DA(hactiv)
        else:
            raise ValueError("Which transformation for DAout ?")

        # deltahebb has shape BS x HS x HS
        # Each row of hebb contain the input weights to a neuron
        deltahebb =  torch.bmm(hactiv.view(BATCHSIZE, HS, 1), hidden.view(BATCHSIZE, 1, HS)) # batched outer product...should it be other way round?


        hebb1 = torch.clamp(hebb + DAout.view(BATCHSIZE, 1, 1) * deltahebb, min=-1.0, max=1.0)
        if self.params['fm'] == 0:
            # Non-modulated part
            hebb2 = torch.clamp(hebb + self.eta * deltahebb, min=-1.0, max=1.0)
        # Soft Clamp (note that it's different from just putting a tanh on top of a freely varying value):
        #hebb1 = torch.clamp( hebb +  torch.clamp(DAout.view(BATCHSIZE, 1, 1) * deltahebb, min=0.0) * (1 - hebb) +
        #        torch.clamp(DAout.view(BATCHSIZE, 1, 1)  * deltahebb, max=0.0) * (hebb + 1) , min=-1.0, max=1.0)
        #hebb2 = torch.clamp( hebb +  torch.clamp(self.eta * deltahebb, min=0.0) * (1 - hebb) +  torch.clamp(self.eta * deltahebb, max=0.0) * (hebb + 1) , min=-1.0, max=1.0)
        # Purely additive, no clamping. This will almost certainly diverge, don't use it!
        #hebb1 = hebb + DAout.view(BATCHSIZE, 1, 1) * deltahebb
        #hebb2 = hebb + self.eta * deltahebb

        if self.params['fm'] == 1:
            hebb = hebb1
        elif self.params['fm'] == 0:
            # Combine the modulated and non-modulated part
            hebb = torch.cat( (hebb1[:, :self.params['hs']//2, :], hebb2[:,  self.params['hs'] // 2:, :]), dim=1) # Maybe along dim=2 instead?...
        else:
            raise ValueError("Must select whether fully modulated or not (params['fm'])")

        hidden = hactiv

        return activout, valueout, DAout, hidden, hebb

    def initialZeroHebb(self):
        return Variable(torch.zeros(self.params['bs'], self.params['hs'], self.params['hs']) , requires_grad=False)

    def initialZeroState(self):
        BATCHSIZE = self.params['bs']
        return Variable(torch.zeros(BATCHSIZE, self.params['hs']), requires_grad=False )





class RetroModulRNN(nn.Module):
    def __init__(self, params):
        super(RetroModulRNN, self).__init__()
        # NOTE: 'outputsize' excludes the value and neuromodulator outputs!
        for paramname in ['outputsize', 'inputsize', 'hs', 'bs', 'fm']:
            if paramname not in params.keys():
                raise KeyError("Must provide missing key in argument 'params': "+paramname)
        NBDA = 1 # For now we limit the number of neuromodulatory-output neurons to 1
        # Doesn't work with our version of PyTorch:
        #self.device = torch.device("cuda:0" if self.params['device'] == 'gpu' else "cpu")
        self.params = params
        self.activ = F.tanh
        self.i2h = torch.nn.Linear(self.params['inputsize'], params['hs'])
        self.w =  torch.nn.Parameter((.01 * torch.t(torch.rand(params['hs'], params['hs']))), requires_grad=True)
        self.alpha =  torch.nn.Parameter((.01 * torch.t(torch.rand(params['hs'], params['hs']))), requires_grad=True)
        self.eta = torch.nn.Parameter((.1 * torch.ones(1)), requires_grad=True)  # Everyone has the same eta (only for the non-modulated part, if any!)
        self.etaet = torch.nn.Parameter((.1 * torch.ones(1)), requires_grad=True)  # Everyone has the same etaet
        self.h2DA = torch.nn.Linear(params['hs'], NBDA)
        self.h2o = torch.nn.Linear(params['hs'], self.params['outputsize'])
        self.h2v = torch.nn.Linear(params['hs'], 1)

    def forward(self, inputs, hidden, hebb, et, pw):
            NBDA = 1
            BATCHSIZE = self.params['bs']
            HS = self.params['hs']

            hactiv = self.activ(self.i2h(inputs).view(BATCHSIZE, HS, 1) + torch.matmul((self.w + torch.mul(self.alpha, pw)),
                            hidden.view(BATCHSIZE, HS, 1))).view(BATCHSIZE, HS)
            activout = self.h2o(hactiv)  # Pure linear, raw scores - will be softmaxed later
            valueout = self.h2v(hactiv)

            # Now computing the Hebbian updates...

            # With batching, DAout is a matrix of size BS x 1 (Really BS x NBDA, but we assume NBDA=1 for now in the deltahebb multiplication below)
            if self.params['da'] == 'tanh':
                DAout = F.tanh(self.h2DA(hactiv))
            elif self.params['da'] == 'sig':
                DAout = F.sigmoid(self.h2DA(hactiv))
            elif self.params['da'] == 'lin':
                DAout =  self.h2DA(hactiv)
            else:
                raise ValueError("Which transformation for DAout ?")

            if self.params['rule'] == 'hebb':
                deltahebb =  torch.bmm(hactiv.view(BATCHSIZE, HS, 1), hidden.view(BATCHSIZE, 1, HS)) # batched outer product...should it be other way round?
            elif self.params['rule'] == 'oja':
                deltahebb =  torch.mul(hactiv.view(BATCHSIZE, HS, 1), (hidden.view(BATCHSIZE, 1, HS) - torch.mul(self.w.view(1, HS, HS), hactiv.view(BATCHSIZE, HS, 1))))
            else:
                raise ValueError("Must specify learning rule ('hebb' or 'oja')")

            # Hard clamp
            deltapw = DAout.view(BATCHSIZE,1,1) * et
            pw1 = torch.clamp(pw + deltapw, min=-1.0, max=1.0)

            # Should we have a fully neuromodulated network, or only half?
            if self.params['fm'] == 1:
                pw = pw1
            elif self.params['fm']==0:
                hebb = torch.clamp(hebb + self.eta * deltahebb, min=-1.0, max=1.0)
                pw = torch.cat( (hebb[:, :self.params['hs']//2, :], pw1[:,  self.params['hs'] // 2:, :]), dim=1) # Maybe along dim=2 instead?...
            else:
                raise ValueError("Must select whether fully modulated or not")

            # Updating the eligibility trace - always a simple decay term.
            # Note that self.etaet != self.eta (which is used for hebb, i.e. the non-modulated part)
            deltaet = deltahebb
            et = (1 - self.etaet) * et + self.etaet *  deltaet

            hidden = hactiv
            return activout, valueout, DAout, hidden, hebb, et, pw




    def initialZeroHebb(self):
        return Variable(torch.zeros(self.params['bs'], self.params['hs'], self.params['hs']) , requires_grad=False)

    def initialZeroPlasticWeights(self):
        return Variable(torch.zeros(self.params['bs'], self.params['hs'], self.params['hs']) , requires_grad=False)
    def initialZeroState(self):
        return Variable(torch.zeros(self.params['bs'], self.params['hs']), requires_grad=False )


In [0]:
# Stimulus-response task as described in Miconi et al. ICLR 2019.
t
# Copyright (c) 2018-2019 Uber Technologies, Inc.
#
# Licensed under the Uber Non-Commercial License (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at the root directory of this project.

import argparse
import pdb
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
from numpy import random
import torch.nn.functional as F
from torch import optim
from torch.optim import lr_scheduler
import random
import sys
import pickle
import time
import os
import platform
#import makemaze

import numpy as np
#import matplotlib.pyplot as plt
import glob





np.set_printoptions(precision=4)



ADDINPUT = 4 # 1 inputs for the previous reward, 1 inputs for numstep, 1 unused,  1 "Bias" inputs


def train(paramdict):
    #params = dict(click.get_current_context().params)

    #params['inputsize'] =  RFSIZE * RFSIZE + ADDINPUT + NBNONRESTACTIONS
    print("Starting training...")
    params = {}
    #params.update(defaultParams)
    params.update(paramdict)
    print("Passed params: ", params)
    print(platform.uname())
    #params['nbsteps'] = params['nbshots'] * ((params['prestime'] + params['interpresdelay']) * params['nbclasses']) + params['prestimetest']  # Total number of steps per episode
    suffix = "SRB_"+"".join([str(x)+"_" if pair[0] != 'pe' and pair[0] != 'nbsteps' and pair[0] != 'rngseed' and pair[0] != 'save_every' and pair[0] != 'test_every' else '' for pair in sorted(zip(params.keys(), params.values()), key=lambda x:x[0] ) for x in pair])[:-1] + "_rngseed_" + str(params['rngseed'])   # Turning the parameters into a nice suffix for filenames
    print(suffix)

    #NBINPUTBITS = params['ni'] + 1
    NBINPUTBITS = params['cs'] + 1 # The additional bit is for the response cue (i.e. the "Go" cue)
    params['outputsize'] =  2  # "response" and "no response"
    params['inputsize'] = NBINPUTBITS +  params['outputsize'] + ADDINPUT  # The total number of input bits is the size of inputs, plus the "response cue" input, plus the number of actions, plus the number of additional inputs

    # This doesn't work with our version of PyTorch
    #params['device'] = 'gpu'
    #device = torch.device("cuda:0" if self.params['device'] == 'gpu' else "cpu")
    BS = params['bs']

    # Initialize random seeds (first two redundant?)
    print("Setting random seeds")
    np.random.seed(params['rngseed']); random.seed(params['rngseed']); torch.manual_seed(params['rngseed'])

    print("Initializing network")
    if params['type'] == 'modul':
        net = RetroModulRNN(params)
    elif params['type'] == 'modplast':
        net = SimpleModulRNN(params)
    elif params['type'] == 'plastic':
        net = PlasticRNN(params)
    elif params['type'] == 'rnn':
        net = NonPlasticRNN(params)
    net.to(device)

    print ("Shape of all optimized parameters:", [x.size() for x in net.parameters()])
    allsizes = [torch.numel(x.data.cpu()) for x in net.parameters()]
    print ("Size (numel) of all optimized elements:", allsizes)
    print ("Total size (numel) of all optimized elements:", sum(allsizes))

    #total_loss = 0.0
    print("Initializing optimizer")
    #optimizer = torch.optim.SGD(net.parameters(), lr=1.0*params['lr'], weight_decay=params['l2'])
    #optimizer = torch.optim.RMSprop(net.parameters(), lr=1.0*params['lr'], weight_decay=params['l2'])
    optimizer = torch.optim.Adam(net.parameters(), lr=1.0*params['lr'], eps=params['eps'], weight_decay=params['l2'])
    #optimizer = torch.optim.Adam(net.parameters(), lr=1.0*params['lr'], eps=1e-4, weight_decay=params['l2'])
    #optimizer = torch.optim.SGD(net.parameters(), lr=1.0*params['lr'])
    #scheduler = torch.optim.lr_scheduler.StepLR(optimizer, gamma=params['gamma'], step_size=params['steplr'])

    #LABSIZE = params['lsize']
    #lab = np.ones((LABSIZE, LABSIZE))
    #CTR = LABSIZE // 2

    # Simple cross maze
    #lab[CTR, 1:LABSIZE-1] = 0
    #lab[1:LABSIZE-1, CTR] = 0


    # Double-T maze
    #lab[CTR, 1:LABSIZE-1] = 0
    #lab[1:LABSIZE-1, 1] = 0
    #lab[1:LABSIZE-1, LABSIZE - 2] = 0

    # Grid maze
    #lab[1:LABSIZE-1, 1:LABSIZE-1].fill(0)
    #for row in range(1, LABSIZE - 1):
    #    for col in range(1, LABSIZE - 1):
    #        if row % 2 == 0 and col % 2 == 0:
    #            lab[row, col] = 1
    #lab[CTR,CTR] = 0 # Not strictly necessary, but perhaps helps loclization by introducing a detectable irregularity in the center


    #LABSIZE = params['msize']
    #lab = np.ones((LABSIZE, LABSIZE))
    #CTR = LABSIZE // 2


    ## Grid maze
    #lab[1:LABSIZE-1, 1:LABSIZE-1].fill(0)
    #for row in range(1, LABSIZE - 1):
    #    for col in range(1, LABSIZE - 1):
    #        if row % 2 == 0 and col % 2 == 0:
    #            lab[row, col] = 1
    #lab[CTR,CTR] = 0 # Not strictly necessary, but perhaps helps loclization by introducing a detectable irregularity in the center

    if params['saved_already'] == False:
        # create dictionary to keep track of stats over all episodes
        all_losses_objective = []
        all_total_rewards = []
        epoch=0
    else:
        checkpoint_path = 'saved/Checkpoint' + params['net_type'] + '.pt'
        checkpoint = torch.load(checkpoint_path)
        print('Loading states 1')
        print(net.load_state_dict(checkpoint['state_dict']))
        print('Loading states 2')
        print(optimizer.load_state_dict(checkpoint['optimizer']))
        epoch = checkpoint['iters'] + 1
        stats=checkpoint['stats_dict'] 
        all_total_rewards = stats['total_rewards']
        all_losses_objective = stats['losses_objectives']
        print(f"Loaded from '{checkpoint_path}' at episode {epoch}")

    all_losses = []
    all_grad_norms = []
    all_losses_v = []
    lossbetweensaves = 0
    nowtime = time.time()
    #meanreward = np.zeros((LABSIZE, LABSIZE))
    meanreward = np.zeros(params['ni'])
    meanrewardT = np.zeros((params['ni'], params['eplen']))

    nbtrials = [0]*BS
    totalnbtrials = 0
    nbtrialswithcc = 0


    print("Starting episodes!")

    for numepisode in tqdm(range(epoch, params['nbiter'])):

        PRINTTRACE = 0
        #if (numepisode+1) % (1 + params['pe']) == 0:
        if (numepisode+1) % (params['pe']) == 0:
            PRINTTRACE = 1

        #lab = makemaze.genmaze(size=LABSIZE, nblines=4)
        #count = np.zeros((LABSIZE, LABSIZE))

        # # Select the reward location for this episode - not on a wall!
        # rposr = 0; rposc = 0
        # while lab[rposr, rposc] == 1:
        #     rposr = np.random.randint(1, LABSIZE - 1)
        #     rposc = np.random.randint(1, LABSIZE - 1)

        # # We always start the episode from the center (when hitting reward, we may teleport either to center or to a random location depending on params['rsp'])
        # posc = CTR
        # posr = CTR




        optimizer.zero_grad()
        loss = 0
        lossv = 0
        hidden = net.initialZeroState().to(device)
        if params['type'] != 'rnn':
            hebb = net.initialZeroHebb().to(device)
        if params['type'] == 'modul':
            et = net.initialZeroHebb().to(device) # Eligibility Trace is identical to Hebbian Trace in shape
            pw = net.initialZeroPlasticWeights().to(device)
        numactionchosen = 0


        # Generate the cues. Make sure they're all different (important when using very small cues for debugging, e.g. cs=2, ni=2)
        cuedata=[]
        for nb in range(BS):
            cuedata.append([])
            for ncue in range(params['ni']):
                assert len(cuedata[nb]) == ncue
                foundsame = 1
                cpt = 0
                while foundsame > 0 :
                    cpt += 1
                    if cpt > 10000:
                        # This should only occur with very weird parameters, e.g. cs=2, ni>4
                        raise ValueError("Could not generate a full list of different cues")
                    foundsame = 0
                    candidate = np.random.randint(2, size=params['cs']) * 2 - 1
                    for backtrace in range(ncue):
                        if np.array_equal(cuedata[nb][backtrace], candidate):
                            foundsame = 1

                cuedata[nb].append(candidate)


        reward = np.zeros(BS)
        sumreward = np.zeros(BS)
        rewards = []
        vs = []
        logprobs = []
        cues=[]
        for nb in range(BS):
            cues.append([])
        dist = 0
        numactionschosen = np.zeros(BS, dtype='int32')

        #reward = 0.0
        #rewards = []
        #vs = []
        #logprobs = []
        #sumreward = 0.0
        nbtrials = np.zeros(BS)
        nbrewardabletrials = np.zeros(BS)
        thistrialhascorrectcue = np.zeros(BS)
        triallength = np.zeros(BS, dtype='int32')
        correctcue = np.random.randint(params['ni'], size=BS)
        trialstep = np.zeros(BS, dtype='int32')

        #print("EPISODE ", numepisode)
        for numstep in range(params['eplen']):

            #if params['clamp'] == 0:
            inputs = np.zeros((BS, params['inputsize']), dtype='float32')
            #else:
            #    inputs = np.zeros((1, params['hs']), dtype='float32')

            for nb in range(BS):

                if trialstep[nb] == 0:
                    thistrialhascorrectcue[nb] = 0
                    # Trial length is randomly modulated for each trial; first time step always -1 (i.e. no input cue), last time step always response-cue (i.e. NBINPUTBITS-1).
                    #triallength = params['ni'] // 2  + 3 + np.random.randint(1 + params['ni'])  # 3 fixed-cue time steps (1st, last and next-to-last) + some random nb of no-cue time steps
                    triallength[nb] = params['ni'] // 2  + 3 + np.random.randint(params['ni'])  # 3 fixed-cue time steps (1st, last and next-to-last) + some random nb of no-cue time steps



                    # In any trial, we only show half the cues (randomly chosen), once each:
                    mycues = [x for x in range(params['ni'])]
                    random.shuffle(mycues); mycues = mycues[:len(mycues) // 2]
                    # The rest is filled with no-input time steps (i.e. cue = -1), but also with the 3 fixed-cue steps (1st, last, next-to-last)
                    for nc in range(triallength[nb] - 3  - len(mycues)):
                        mycues.append(-1)
                    random.shuffle(mycues)
                    mycues.insert(0, -1); mycues.append(params['ni']); mycues.append(-1)  # The first and last time step have no input (cue -1), the next-to-last has the response cue.
                    assert(len(mycues) == triallength[nb])
                    cues[nb] = mycues


                inputs[nb, :NBINPUTBITS] = 0
                if cues[nb][trialstep[nb]] > -1 and cues[nb][trialstep[nb]] < params['ni']:
                    #inputs[0, cues[trialstep]] = 1.0
                    inputs[nb, :NBINPUTBITS-1] = cuedata[nb][cues[nb][trialstep[nb]]][:]
                    if cues[nb][trialstep[nb]] == correctcue[nb]:
                        thistrialhascorrectcue[nb] = 1
                if cues[nb][trialstep[nb]] == params['ni']:
                    inputs[nb, NBINPUTBITS-1] = 1  # "Go" cue


                inputs[nb, NBINPUTBITS + 0] = 1.0 # Bias neuron, probably not necessary
                inputs[nb,NBINPUTBITS +  1] = numstep / params['eplen']
                inputs[nb, NBINPUTBITS + 2] = 1.0 * reward[nb] # Reward from previous time step
                if numstep > 0:
                    inputs[nb, NBINPUTBITS + ADDINPUT + numactionschosen[nb]] = 1  # Previously chosen action

            inputsC = torch.from_numpy(inputs).to(device)
            # Might be better:
            #if rposr == posr and rposc = posc:
            #    inputs[0][-4] = 100.0
            #else:
            #    inputs[0][-4] = 0

            # Running the network

            ## Running the network
            if params['type'] == 'modplast':
                y, v, DAout, hidden, hebb = net(Variable(inputsC, requires_grad=False).to(device), hidden, hebb)  # y  should output raw scores, not probas
            elif params['type'] == 'modul':
                y, v, DAout, hidden, hebb, et, pw  = net(Variable(inputsC, requires_grad=False).to(device), hidden, hebb, et, pw)  # y  should output raw scores, not probas
            elif params['type'] == 'plastic':
                y, v, hidden, hebb = net(Variable(inputsC, requires_grad=False).to(device), hidden, hebb)  # y  should output raw scores, not probas
            elif params['type'] == 'rnn':
                y, v, hidden = net(Variable(inputsC, requires_grad=False).to(device), hidden)  # y  should output raw scores, not probas
            else:
                raise ValueError("Network type unknown or not yet implemented!")



            y = F.softmax(y, dim=1)
            # Must convert y to probas to use this !
            distrib = torch.distributions.Categorical(y)
            actionschosen = distrib.sample()
            logprobs.append(distrib.log_prob(actionschosen))
            numactionschosen = actionschosen.data.cpu().numpy()    # Turn to scalar

            if PRINTTRACE:
                print("Step ", numstep, " Inputs (1st in batch): ", inputs[0,:params['inputsize']], " - Outputs(0): ", y.data.cpu().numpy()[0,:], " - action chosen(0): ", numactionschosen[0],
                        "TrialLen(0):", triallength[0], "trialstep(0):", trialstep[0], "TTHCC(0): ", thistrialhascorrectcue[0], " -Reward (previous step): ", reward[0], ", cues(0):", cues[0], ", cc(0):", correctcue[0])

                #print("Step ", numstep, " Inputs: ", inputs[0,:params['inputsize']], " - Outputs: ", y.data.cpu().numpy(), " - action chosen: ", numactionchosen,
                #        " - mean abs pw: ", np.mean(np.abs(pw.data.cpu().numpy())), "TrialLen:", triallength, "trialstep:", trialstep, "TTHCC: ", thistrialhascorrectcue, " -Reward (previous step): ", reward, ", cues:", cues, ", cc:", correctcue)

            reward = np.zeros(BS, dtype='float32')

            for nb in range(BS):
                if numactionschosen[nb] == 1:
                    # Small penalty for any non-rest action taken
                    reward[nb]  -= params['wp']


            ### DEBUGGING
            ## Easiest possible episode-dependent response (i.e. the easiest
            ## possible problem that actually require meta-learning, with ni=2)
            ## This one works pretty wel... But harder ones don't work well!
            #if numactionchosen == correctcue :
            #        reward = params['rew']
            #else:
            #        reward = -params['rew']


                trialstep[nb] += 1
                if trialstep[nb] == triallength[nb] - 1:
                    # This was the next-to-last step of the trial (and we showed the response signal, unless it was the first few steps in episode).
                    assert(cues[nb][trialstep[nb] - 1] == params['ni'] or numstep < 2)
                    # We must deliver reward (which will be perceived by the agent at the next step), positive or negative, depending on response
                    if thistrialhascorrectcue[nb] and numactionschosen[nb] == 1:
                        reward[nb] += params['rew']
                    elif (not thistrialhascorrectcue[nb]) and numactionschosen[nb] == 0:
                        reward[nb] += params['rew']
                    else:
                        reward[nb] -= params['rew']

                    if np.random.rand() < params['pf']:
                        reward[nb] = -reward[nb]

                if trialstep[nb] == triallength[nb]:
                    # This was the last step of the trial (and we showed no input)
                    assert(cues[nb][trialstep[nb] - 1] == -1 or numstep < 2)
                    nbtrials[nb] += 1
                    totalnbtrials += 1
                    if thistrialhascorrectcue[nb]:
                        nbtrialswithcc += 1
                        #nbrewardabletrials += 1
                    # Trial is dead, long live trial
                    trialstep[nb] = 0

                    # We initialize the hidden state between trials!
                    #if params['is'] == 1:
                    #    hidden = net.initialZeroState()



            rewards.append(reward)
            vs.append(v)
            sumreward += reward



            #if params['alg'] in ['A3C' , 'REIE' , 'REIT']:

            loss += (params['bent'] * y.pow(2).sum() / BS )   # We want to penalize concentration, i.e. encourage diversity; our version of PyTorch does not have an entropy() function for Distribution, so we use this instead.



            ##if PRINTTRACE:
            ##    print("Probabilities:", y.data.cpu().numpy(), "Picked action:", numactionchosen, ", got reward", reward)

        R = Variable(torch.zeros(BS), requires_grad=False).to(device)
        gammaR = params['gr']
        for numstepb in reversed(range(params['eplen'])) :
            R = gammaR * R + Variable(torch.from_numpy(rewards[numstepb]), requires_grad=False).to(device)
            ctrR = R - vs[numstepb][0]
            lossv += ctrR.pow(2).sum() / BS
            loss -= (logprobs[numstepb] * ctrR.detach()).sum() / BS  # Need to check if detach() is OK
            #pdb.set_trace()


        # Episode is done, now let's do the actual computations
        #gammaR = params['gr']
        #if params['alg'] == 'A3C':
        #    R = 0
        #    for numstepb in reversed(range(params['eplen'])) :
        #        R = gammaR * R + rewards[numstepb]
        #        lossv += (vs[numstepb][0] - R).pow(2)
        #        loss -= logprobs[numstepb] * (R - vs[numstepb].data[0][0])  # Not sure if the "data" is needed... put it b/c of worry about weird gradient flows
        #    loss += params['bv'] * lossv

        #elif params['alg'] in ['REI', 'REIE']:
        #    R = sumreward
        #    baseline = meanreward[correctcue]
        #    for numstepb in reversed(range(params['eplen'])) :
        #        loss -= logprobs[numstepb] * (R - baseline)
        #elif params['alg'] == 'REIT':
        #    R = 0
        #    for numstepb in reversed(range(params['eplen'])) :
        #        R = gammaR * R + rewards[numstepb]
        #        loss -= logprobs[numstepb] * (R - meanrewardT[correctcue, numstepb])
        #else:
        #    raise ValueError("Must select algo type")
        #elif params['alg'] == 'REINOB':
        #    R = sumreward
        #    for numstepb in reversed(range(params['eplen'])) :
        #        loss -= logprobs[numstepb] * R
        #elif params['alg'] == 'REITMP':
        #    R = 0
        #    for numstepb in reversed(range(params['eplen'])) :
        #        R = gammaR * R + rewards[numstepb]
        #        loss -= logprobs[numstepb] * R

        #else:
        #    raise ValueError("Which algo?")

        #meanreward[correctcue] = (1.0 - params['nu']) * meanreward[correctcue] + params['nu'] * sumreward
        ##meanreward[rposr, rposc] = (1.0 - params['nu']) * meanreward[rposr, rposc] + params['nu'] * sumreward
        #R = 0
        #for numstepb in reversed(range(params['eplen'])) :
        #    R = gammaR * R + rewards[numstepb]
        #    meanrewardT[correctcue, numstepb] = (1.0 - params['nu']) * meanrewardT[correctcue, numstepb] + params['nu'] * R

        loss += params['blossv'] * lossv
        loss /= params['eplen']

        if PRINTTRACE:
            #if params['alg'] == 'A3C':
            print("lossv: ", float(lossv))
            #elif params['alg'] in ['REI', 'REIE', 'REIT']:
            #    print("meanreward baselines: ", [meanreward[x] for x in range(params['ni'])])
            print ("Total reward for this episode(0):", sumreward[0], "Prop. of trials w/ rewarded cue:", (nbtrialswithcc / totalnbtrials))
            print("Nb trials for this episode(0):", nbtrials[0], "[2]:",nbtrials[2]," Total Nb of trials:", totalnbtrials)

        #if params['squash'] == 1:
        #    if sumreward < 0:
        #        sumreward = -np.sqrt(-sumreward)
        #    else:
        #        sumreward = np.sqrt(sumreward)
        #elif params['squash'] == 0:
        #    pass
        #else:
        #    raise ValueError("Incorrect value for squash parameter")

        #loss *= sumreward

        #for p in net.parameters():
        #    p.grad.data.clamp_(-params['clamp'], params['clamp'])
        loss.backward()
        all_grad_norms.append(torch.nn.utils.clip_grad_norm(net.parameters(), params['gc']))
        #if numepisode > 100:  # Burn-in period for meanreward
        optimizer.step()


        #print(sumreward)
        lossnum = float(loss)
        lossbetweensaves += lossnum
        all_losses_objective.append(lossnum)
        all_total_rewards.append(sumreward.mean())
        #all_total_rewards.append(sumreward[0])
            #all_losses_v.append(lossv.data[0])
        #total_loss  += lossnum


        if (numepisode+1) % params['pe'] == 0:

            print(numepisode, "====")
            print("Mean loss: ", lossbetweensaves / params['pe'])
            lossbetweensaves = 0
            print("Mean reward: ", np.sum(all_total_rewards[-params['pe']:])/ params['pe'])
            previoustime = nowtime
            nowtime = time.time()
            print("Time spent on last", params['pe'], "iters: ", nowtime - previoustime)
            if params['type'] == 'plastic' or params['type'] == 'lstmplastic':
                print("ETA: ", float(net.eta), "alpha[0,1]: ", net.alpha.data.cpu().numpy()[0,1], "w[0,1]: ", net.w.data.cpu().numpy()[0,1] )
            elif params['type'] == 'modul' or params['type'] == 'modul2':
                print("ETA: ", net.eta.data.cpu().numpy(), " etaet: ", net.etaet.data.cpu().numpy(), " mean-abs pw: ", np.mean(np.abs(pw.data.cpu().numpy())))
            elif params['type'] == 'rnn':
                print("w[0,1]: ", net.w.data.cpu().numpy()[0,1] )

        if (numepisode+1) % params['save_every'] == 0:
            stats = {
                'total_rewards':all_total_rewards,
                'losses_objectives':all_losses_objective
            }
            state = {
                'iters': numepisode,
                'state_dict': net.state_dict(),
                'optimizer': optimizer.state_dict(),
                'stats_dict': stats
            }
            savepath='saved/Checkpoint'+params['net_type']+'.pt'
            torch.save(state,savepath)
            print(f"Saved to '{savepath}' at episode {numepisode}")

    stats = {
        'total_rewards':all_total_rewards,
        'losses_objectives':all_losses_objective
    }
    return {
          'iters': params['nbiter'],
          'state_dict': net.state_dict(),
          'optimizer': optimizer.state_dict(),
          'stats_dict': stats
    }



In [7]:
params = {
    'net_type': 'RM', # for saving
    'type' : 'modul',
    'lr': 1e-4,
    'eplen': 120,
    'hs': 200,
    'l2':0,
    'pe':10000,
    'bv':0.1,
    'blossv':0.1,
    'bent':0.1,
    'rew':1,
    'wp':0,
    'save_every':1000,
    'da':'tanh',
    'clamp':0,
    'nbiter':60000,
    'fm':1,
    'ni':4,
    'pf':0.0,
    'alg':'A3C',
    'rule':'hebb',
    'cs':20,
    'eps':1e-6,
    'is':0,
    'bs':30,
    'gr':0.9,
    'gc':2.0,
    'rngseed':0,
    'saved_already':True
}


save = train(params)
savepath='saved/Final'+params['net_type']+'.pt'
torch.save(save,savepath)
print("Saved!")

Starting training...
Passed params:  {'net_type': 'RM', 'type': 'modul', 'lr': 0.0001, 'eplen': 120, 'hs': 200, 'l2': 0, 'pe': 10000, 'bv': 0.1, 'blossv': 0.1, 'bent': 0.1, 'rew': 1, 'wp': 0, 'save_every': 1000, 'da': 'tanh', 'clamp': 0, 'nbiter': 60000, 'fm': 1, 'ni': 4, 'pf': 0.0, 'alg': 'A3C', 'rule': 'hebb', 'cs': 20, 'eps': 1e-06, 'is': 0, 'bs': 30, 'gr': 0.9, 'gc': 2.0, 'rngseed': 0, 'saved_already': True}
uname_result(system='Linux', node='7a627b1ff775', release='4.19.104+', version='#1 SMP Wed Feb 19 05:26:34 PST 2020', machine='x86_64', processor='x86_64')
SRB_alg_A3C_bent_0.1_blossv_0.1_bs_30_bv_0.1_clamp_0_cs_20_da_tanh_eplen_120_eps_1e-06_fm_1_gc_2.0_gr_0.9_hs_200_is_0_l2_0_lr_0.0001_nbiter_60000_net_type_RM_ni_4_pf_0.0_rew_1_rule_hebb_saved_already_True_type_modul_wp_0_rngseed_0
Setting random seeds
Initializing network
Shape of all optimized parameters: [torch.Size([200, 200]), torch.Size([200, 200]), torch.Size([1]), torch.Size([1]), torch.Size([200, 27]), torch.Size([20

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1558: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:471: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


Saved to 'saved/CheckpointRM.pt' at episode 32999
Saved to 'saved/CheckpointRM.pt' at episode 33999
Saved to 'saved/CheckpointRM.pt' at episode 34999
Saved to 'saved/CheckpointRM.pt' at episode 35999
Saved to 'saved/CheckpointRM.pt' at episode 36999
Saved to 'saved/CheckpointRM.pt' at episode 37999
Saved to 'saved/CheckpointRM.pt' at episode 38999
Step  0  Inputs (1st in batch):  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0.]  - Outputs(0):  [0.5065 0.4935]  - action chosen(0):  1 TrialLen(0): 8 trialstep(0): 0 TTHCC(0):  0.0  -Reward (previous step):  0.0 , cues(0): [-1, -1, -1, 0, -1, 2, 4, -1] , cc(0): 1
Step  1  Inputs (1st in batch):  [0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     1.     0.0083 0.     0.     0.     1.    ]  - Outputs(0):  [0.5063 0.4937]  - action chosen(0):  1 TrialLen(0): 8 trialstep(0): 1 TTHCC(0):  0.0  -Reward (previous step):  0